In [1]:
import glob
from datasets import load_dataset, Image, Dataset

In [2]:
labels = []
image_paths = glob.glob('/Users/amineelqaraoui/Desktop/CIH_CLASSIFIED/*.jpg')

for image_path in image_paths:
    
    f = open(f"/Users/amineelqaraoui/Desktop/CIH_CLASSIFIED/{image_path.split('/')[-1].split('.jpg')[0]}_WITHQRCODE.txt", 'r', encoding="utf-8")

    labels.append(f.read())

In [3]:
image_paths

['/Users/amineelqaraoui/Desktop/CIH_CLASSIFIED/63.jpg',
 '/Users/amineelqaraoui/Desktop/CIH_CLASSIFIED/189.jpg',
 '/Users/amineelqaraoui/Desktop/CIH_CLASSIFIED/77.jpg',
 '/Users/amineelqaraoui/Desktop/CIH_CLASSIFIED/162.jpg',
 '/Users/amineelqaraoui/Desktop/CIH_CLASSIFIED/176.jpg',
 '/Users/amineelqaraoui/Desktop/CIH_CLASSIFIED/88.jpg',
 '/Users/amineelqaraoui/Desktop/CIH_CLASSIFIED/348.jpg',
 '/Users/amineelqaraoui/Desktop/CIH_CLASSIFIED/360.jpg',
 '/Users/amineelqaraoui/Desktop/CIH_CLASSIFIED/374.jpg',
 '/Users/amineelqaraoui/Desktop/CIH_CLASSIFIED/228.jpg',
 '/Users/amineelqaraoui/Desktop/CIH_CLASSIFIED/214.jpg',
 '/Users/amineelqaraoui/Desktop/CIH_CLASSIFIED/200.jpg',
 '/Users/amineelqaraoui/Desktop/CIH_CLASSIFIED/201.jpg',
 '/Users/amineelqaraoui/Desktop/CIH_CLASSIFIED/215.jpg',
 '/Users/amineelqaraoui/Desktop/CIH_CLASSIFIED/229.jpg',
 '/Users/amineelqaraoui/Desktop/CIH_CLASSIFIED/375.jpg',
 '/Users/amineelqaraoui/Desktop/CIH_CLASSIFIED/361.jpg',
 '/Users/amineelqaraoui/Desktop/CI

In [4]:
labels

["BORDEREAU DE RETRAIT BORDEREAU\nReference d'operation:029400191159708042022\nNom du client  :AZOUA HAFIDA\nNumero de compte :2447596211029400\nBeneficiaire:AZOUA HAFIDA\nMontant du retrait  :400.00 MAD",
 'BORDEREAU DE VERSEMENT SUR COMPTE\nReference operation:034200231521608042022\nNumero de compte:3857896211017200\nMontant du versement:26 960.00 MAD\nVerse par :TAFILALT MERYEM',
 "BORDEREAU DE VERSEMENT SUR COMPTE SUR CARNET\nReference d'operation :029400211121908042022\nMontant : 3 150.00 MAD\nVerse par : SBAI FAHIM AHMED\nNumero du compte : 3011025231008100\nIntitule du compte : MONSIEUR AHMED SBAI FAHIM",
 'BORDEREAU DE VERSEMENT SUR COMPTE\nReference operation:029400237216608042022\nNumero de compte:2621594211029400\nMontant du versement:1 000.00 MAD\nVerse par :EL FANAN MOHAMED',
 'BORDEREAU DE VERSEMENT SUR COMPTE\nReference operation:029400235920408042022\nNumero de compte:2754817211015900\nMontant du versement:15 000.00 MAD\nVerse par :CHANTOUF HAJAR',
 'BORDEREAU DE VERSEM

In [5]:
train_dataset = Dataset.from_dict({"image": image_paths, "label": labels}).cast_column("image", Image())

In [6]:
eval_dataset = Dataset.from_dict({"image": image_paths, "label": labels}).cast_column("image", Image())

In [7]:
def transforms(examples):
    
    examples["pixel_values"] = [feature_extractor(img, return_tensors="pt").pixel_values for img in examples["image"]]
    del examples["image"]
    return examples

In [8]:
train_dataset = train_dataset.with_transform(transforms)
eval_dataset = eval_dataset.with_transform(transforms)

In [9]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(eval_dataset, batch_size=8)

In [10]:
from transformers import ViTFeatureExtractor, BertTokenizer, VisionEncoderDecoderModel
from datasets import load_dataset

feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(
    "google/vit-base-patch16-224-in21k", "bert-base-uncased"
)

model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.7.crossattention.self.query.weight', 'bert.encoder.layer.1.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.value.weight', 'bert.encoder.layer.10.crossattention.output.LayerNorm.bias', 'bert.en

In [11]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [12]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [13]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0): ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, 

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_epochs):
    
    model.train()
    
    for step, batch in enumerate(train_dataloader):
        
        batch = {k: v for k, v in batch.items()}
        
        labels = tokenizer(batch['label'], return_tensors="pt", padding=True).input_ids
        
        outputs = model(pixel_values=batch['pixel_values'].squeeze(1), labels=labels)
        
        print(f"Epoch {epoch} Step {step}/{len(train_dataloader)} : Training Loss {outputs.loss}")
        
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        
    
    model.eval()
    
    eval_loss = 0
    
    for batch in eval_dataloader:
        
        batch = {k: v for k, v in batch.items()}
        
        labels = tokenizer(batch['label'], return_tensors="pt", padding=True).input_ids
        
        outputs = model(pixel_values=batch['pixel_values'].squeeze(1), labels=labels)
        
        eval_loss += outputs.loss
        
    eval_loss /= len(eval_dataloader)
        
    print(f"Epoch {epoch} : Validation Loss {eval_loss}")

  0%|          | 0/147 [00:00<?, ?it/s]

Epoch 0 Step 0/49 : Training Loss 12.809100151062012
Epoch 0 Step 1/49 : Training Loss 9.717050552368164
Epoch 0 Step 2/49 : Training Loss 9.304656028747559
Epoch 0 Step 3/49 : Training Loss 8.40674877166748
Epoch 0 Step 4/49 : Training Loss 8.30640983581543
Epoch 0 Step 5/49 : Training Loss 7.702091217041016
Epoch 0 Step 6/49 : Training Loss 7.242342472076416
Epoch 0 Step 7/49 : Training Loss 6.841605186462402
Epoch 0 Step 8/49 : Training Loss 6.647266387939453
Epoch 0 Step 9/49 : Training Loss 6.361289024353027
Epoch 0 Step 10/49 : Training Loss 5.986597061157227
Epoch 0 Step 11/49 : Training Loss 6.068516731262207
Epoch 0 Step 12/49 : Training Loss 5.633447170257568
Epoch 0 Step 13/49 : Training Loss 5.430554389953613
Epoch 0 Step 14/49 : Training Loss 5.158962726593018
Epoch 0 Step 15/49 : Training Loss 5.073119640350342
Epoch 0 Step 16/49 : Training Loss 5.208414077758789
Epoch 0 Step 17/49 : Training Loss 4.878696918487549
Epoch 0 Step 18/49 : Training Loss 4.862663745880127
Epoc